# Audible Detection From Tracking Data 
label plays is_audible == True using custom algorithm

---

In [3]:
import pandas as pd
import numpy as np

In [4]:
def graphFeatures(play):

    # Graph Location
    # plt.figure(figsize=(10, 8))
    # scatter = plt.scatter(
    #     play['x'], 
    #     play['y'], 
    #     c=play['frameId'], 
    #     cmap='viridis', 
    #     s=50, 
    #     edgecolor='k'
    # )
    
    # # Add color bar to show progression of 'frameID'
    # cbar = plt.colorbar(scatter)
    # cbar.set_label('FrameID (Time Progression)')
    
    # # Add labels and title
    # plt.xlabel('X-Coordinate')
    # plt.ylabel('Y-Coordinate')
    # plt.title('Location Over Time for QB ')
    # print("GID:" , play['gameId'].unique() , "PID:" , play['playId'].unique())
    # plt.grid(True)
    
    # # Show the plot
    # plt.show()
    
    plt.plot(play['frameId'], play['s'])
    plt.xlabel('frameId')
    plt.ylabel('S')
    plt.title('S vs FrameID for QB ')
    plt.grid(True)
    plt.show()
    
    plt.plot(play['frameId'], play['o'])
    plt.xlabel('frameId')
    plt.ylabel('O')
    plt.title('O vs FrameID for QB ')
    plt.grid(True)
    plt.show()

In [5]:
# Load tracking,plays, and players data
tracking_files = [
                   '/kaggle/input/nfl-big-data-bowl-2025/tracking_week_1.csv',
                   '/kaggle/input/nfl-big-data-bowl-2025/tracking_week_2.csv',
                   '/kaggle/input/nfl-big-data-bowl-2025/tracking_week_3.csv',
                   '/kaggle/input/nfl-big-data-bowl-2025/tracking_week_4.csv',
                   '/kaggle/input/nfl-big-data-bowl-2025/tracking_week_5.csv',
                   '/kaggle/input/nfl-big-data-bowl-2025/tracking_week_6.csv',
                   '/kaggle/input/nfl-big-data-bowl-2025/tracking_week_7.csv',
                   '/kaggle/input/nfl-big-data-bowl-2025/tracking_week_8.csv',
                   '/kaggle/input/nfl-big-data-bowl-2025/tracking_week_9.csv'
]
playerdata_file = '/kaggle/input/nfl-big-data-bowl-2025/players.csv'
plays_data_file = '/kaggle/input/nfl-big-data-bowl-2025/plays.csv'
plays_general = pd.read_csv(plays_data_file)
players = pd.read_csv(playerdata_file)
tracking = pd.read_csv(tracking_files[2])
# tracking = pd.concat((pd.read_csv(file, low_memory = False) for file in tracking_files), ignore_index=True)

In [6]:
# change to all QBs 
qbs = players[players['position'] == 'QB']
qbs_tracking = tracking.merge(qbs, on='nflId', how='inner', suffixes=('', '_dup')) # add tracking data to qb player data
qbs_tracking = qbs_tracking.loc[:, ~qbs_tracking.columns.str.endswith('_dup')] # remove duplicate columns
qbs_presnap_tracking = qbs_tracking[qbs_tracking["frameType"].isin(["BEFORE_SNAP", "SNAP"])] #filter to before and snap frames 

In [7]:
# testing for specific games or plays 
# qbs_presnap_tracking = qbs_presnap_tracking[qbs_presnap_tracking["gameId"] == 2022092600]
# qbs_presnap_tracking = qbs_presnap_tracking[qbs_presnap_tracking["playId"] == 516]
# 1959 plays in week 3 

In [8]:
# Audible Detection Functions 
def calculate_distance(x1, y1, x2, y2):
    return np.sqrt((x2 - x1)**2 + (y2 - y1)**2)

# Locate the snap frame and its (x, y) coordinates
def get_snap_frame_location(play):
    snap_frame = play[play["frameType"] == "SNAP"]
    if snap_frame.empty:
        return None, None
    snap_row = snap_frame.iloc[0]
    return snap_row["x"], snap_row["y"]

# Filter frames near the snap frame location
def filter_frames_near_snap(play, snap_x, snap_y, DISTANCE_THRESHOLD = 2.0):
    play["distance_from_snap"] = calculate_distance(play["x"], play["y"], snap_x, snap_y)
    return play[play["distance_from_snap"] <= DISTANCE_THRESHOLD]

# Check if QB stopped and when
def check_stop_condition(near_snap, STOP_THRESHOLD= 0.14):
    qb_set_frameId = near_snap[(near_snap["s"] <= STOP_THRESHOLD)]
    qb_set_frameId = qb_set_frameId[qb_set_frameId["frameId"] > 30]
    if qb_set_frameId.empty:
        return None
    qb_set_frameId = qb_set_frameId["frameId"].min()

    return qb_set_frameId

# Detect spikes in speed and orientation
def detect_spikes(play, ORIENTATION_AMPLITUDE = 10.0, SPEED_AMPLITUDE = 0.1, ORIENTATION_PERIOD = 10.0):
    speed_diff = play["s"].diff().abs()
    orientation_diff = play["o"].diff(ORIENTATION_PERIOD).abs()

    small_speed_spike = (speed_diff >= SPEED_AMPLITUDE).any()
    orientation_spike = (orientation_diff >= ORIENTATION_AMPLITUDE).any()

    return small_speed_spike, orientation_spike


In [9]:
# NOT IN USE 
def check_stop_duration(near_snap, STOP_THRESHOLD = 0.16, STOP_DURATION = 10):
    near_snap = near_snap[near_snap["frameId"] > 30]
    # Identify frames where speed is below the STOP_THRESHOLD
    below_threshold = near_snap['s'] <= STOP_THRESHOLD

    # Create a group identifier for consecutive frames
    stops = (below_threshold != below_threshold.shift()).cumsum()

    # Count consecutive frames for each group
    stop_durations = below_threshold.groupby(stops).cumsum()

    # Find the first frame in the earliest sequence that meets STOP_DURATION
    set_frames = near_snap[(below_threshold) & (stop_durations >= STOP_DURATION)]
    if not set_frames.empty:
        qb_set_frameId = set_frames['frameId'].iloc[0]
    else:
        qb_set_frameId = None

    return qb_set_frameId

In [10]:
# Function to process each play and detect patterns
def detect_audible(game_id, play_id, play):
    # Step 1: Get snap frame location:
    snap_x, snap_y = get_snap_frame_location(play)
    if snap_x is None or snap_y is None:
        return False

    # Step 2: Check when QB is set:
    near_snap = filter_frames_near_snap(play, snap_x, snap_y)
    if near_snap.empty:
        return False
    
    qb_set_frameId = check_stop_condition(near_snap)
    # print(qb_set_frameId)
    if qb_set_frameId is None:
        return False

    # Step 4: Detect spikes:
    small_speed_spike, orientation_spike = detect_spikes(play[play["frameId"] > qb_set_frameId])

    # Step 5: Check for Audible 
    if (small_speed_spike and orientation_spike):
        return True

    return False

In [11]:
# Main function to process all plays
def detect_all_audibles(qbs_presnap_tracking):

    # Plays by gameId and playId
    plays = qbs_presnap_tracking.groupby(["gameId", "playId"])
    
    matching_results = plays.apply(lambda play: detect_audible(play.name[0], play.name[1], play))

    # Reset index of matching_results to align with the original DataFrame
    matching_results = matching_results.rename("is_audible").reset_index()

    # Merge the results back into the original DataFrame
    qbs_presnap_tracking = qbs_presnap_tracking.merge(matching_results, on=['gameId', 'playId'], how='left')

    return qbs_presnap_tracking

In [24]:
# Test Audible detection
audible_plays = detect_all_audibles(qbs_presnap_tracking)

# display(qbs_presnap_tracking[qbs_presnap_tracking["playId"] == 1045][["o","frameId"]])
# graphFeatures(audible_plays[audible_plays["playId"] == 2043])

unique_plays = audible_plays[(audible_plays["is_audible"] == True) & (audible_plays["frameId"] == 1)]
unique_plays = unique_plays[["gameId","playId","is_audible"]]
# ADD game time
plays_general = plays_general[['gameId', 'playId','gameClock','quarter']]
unique_plays = unique_plays.merge(plays_general, on=['gameId', 'playId'], how= 'left')
pd.set_option("display.max_rows", None)
# display(unique_plays)
# display(audible_plays["is_audible"].describe())


In [25]:
# display(unique_plays)
display(audible_plays["is_audible"].describe())

count     204466
unique         2
top        False
freq      119594
Name: is_audible, dtype: object

In [13]:
# Download result 
# audible_plays.to_csv('output.csv', index=False, chunksize=100000)
